<a href="https://colab.research.google.com/github/prakashreddy57/Personal-Website/blob/main/publicPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI ENHANCED CUSTOMER FEEDBACK ANALYSIS

## In this project we get the reviews from the url and analyse them to provide the final verdict and rate the product using AI

### There are 3 Major tasks
1. Web Scrapping for the reviews (yelp.com)
2. Generating Verdict
3. Rating the product

# 1. WEB SCRAPPING USING BEAUTIFULSOUP

In [ ]:
!pip install requests beautifulsoup4 pandas numpy

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [ ]:
def getReviews(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  regex = re.compile('.*comment.*')
  results = soup.find_all('p', {'class':regex})
  reviews = [result.text for result in results]
  return reviews

In [ ]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I

In [ ]:
def getReviews(url):
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  regex = re.compile('.*comment.*')
  results = soup.find_all('p', {'class':regex})
  reviews = [result.text for result in results]
  return reviews

In [ ]:
getReviews('https://www.yelp.com/biz/mejico-sydney-2')

['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I

# 2. GENERATING VERDICT USING T5 TRANSFORMER MODEL

In [ ]:
! pip install -q transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [ ]:
example = billsum["train"][0]
for key in example:
    print("A key of the example: \"{}\"".format(key))
    print("The value corresponding to the key-\"{}\"\n \"{}\"".format(key, example[key]))

A key of the example: "text"
The value corresponding to the key-"text"
 "The people of the State of California do enact as follows:


SECTION 1.
Section 130350.5 of the Public Utilities Code is amended to read:
130350.5.
(a) In addition to any other tax that it is authorized by law to impose, the Los Angeles County Metropolitan Transportation Authority (MTA) may impose, in compliance with subdivision (b), a transactions and use tax at a rate of 0.5 percent that is applicable in the incorporated and unincorporated areas of the county.
(b) For purposes of the taxing authority set forth in subdivision (a), all of the following apply:
(1) The tax shall be proposed in a transactions and use tax ordinance, that conforms with Chapter 2 (commencing with Section 7261) to Chapter 4 (commencing with Section 7275), inclusive, of the Transactions and Use Tax Law (Part 1.6 (commencing with Section 7251) of Division 2 of the Revenue and Taxation Code), and that is approved by a majority of the entire

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
tokenized_text = tokenizer(example['text'])
for key in tokenized_text:
    print(key)
    print(tokenized_text[key])

Token indices sequence length is longer than the specified maximum sequence length for this model (3771 > 512). Running this sequence through the model will result in indexing errors


input_ids
[37, 151, 13, 8, 1015, 13, 1826, 103, 3, 35, 2708, 38, 6963, 10, 180, 3073, 9562, 1300, 5568, 12778, 2469, 12100, 13, 8, 2575, 412, 17, 173, 2197, 3636, 19, 21012, 12, 608, 10, 12778, 2469, 12100, 5, 41, 9, 61, 86, 811, 12, 136, 119, 1104, 24, 34, 19, 11330, 57, 973, 12, 3, 12436, 6, 8, 3144, 4975, 1334, 20074, 14630, 9293, 41, 329, 3221, 61, 164, 3, 12436, 6, 16, 5856, 28, 27444, 41, 115, 201, 3, 9, 6413, 11, 169, 1104, 44, 3, 9, 1080, 13, 3, 12100, 1093, 24, 19, 5383, 16, 8, 3, 10975, 11, 73, 10975, 844, 13, 8, 5435, 5, 41, 115, 61, 242, 3659, 13, 8, 1104, 53, 5015, 356, 7444, 16, 27444, 41, 9, 201, 66, 13, 8, 826, 1581, 10, 5637, 37, 1104, 1522, 36, 4382, 16, 3, 9, 6413, 11, 169, 1104, 24417, 6, 24, 4535, 7, 28, 8647, 204, 41, 287, 526, 4733, 28, 5568, 9455, 4241, 61, 12, 8647, 314, 41, 287, 526, 4733, 28, 5568, 489, 25988, 201, 13066, 6, 13, 8, 21469, 7, 11, 2048, 5287, 2402, 41, 13725, 3, 15062, 41, 287, 526, 4733, 28, 5568, 9455, 5553, 61, 13, 6022, 204, 13, 8, 19764, 1

In [ ]:
def preprocess_function(examples):

    inputs = ["summarize: " + doc for doc in examples["text"]]

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum['test'][0]['text']

'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 25250.1 of the Health and Safety Code is amended to read:\n25250.1.\n(a) As used in this article, the following terms have the following meaning:\n(1) (A) “Used oil” means all of the following:\n(i) Oil that has been refined from crude oil, or any synthetic\noil,\noil from any source,\nthat has been used, and, as a result of use or as a consequence of extended storage, or spillage, has been contaminated with physical or chemical impurities.\n(ii) Material that is subject to regulation as used oil under Part 279 (commencing with Section 279.1) of Subchapter I of Chapter 1 of Title 40 of the Code of Federal Regulations.\n(B) Examples of used oil are spent lubricating fluids that have been removed from an engine crankcase, transmission, gearbox, or differential of an automobile, bus, truck, vessel, plane, heavy equipment, or machinery powered by an internal combustion engine; industrial oils, including co

In [ ]:
tokenized_billsum['test'][0]['summary']

'Existing law authorizes the Department of Toxic Substances Control to regulate the disposal of hazardous waste, including used oil, and, for those purposes, defines “used oil” to mean oil that has been refined from crude oil, or any synthetic oil, that has been used, and, as a result of use or as a consequence of extended storage, or spillage, has been contaminated with physical or chemical impurities.\nThis bill would clarify that the synthetic oil referred to in the definition of “used oil” may be from any source.'

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [ ]:
! pip install -q evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_t5_small_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.815565,0.125500,0.036500,0.107200,0.107100,19.000000
2,No log,2.607462,0.134800,0.044200,0.111700,0.111500,19.000000
3,No log,2.545618,0.136200,0.046600,0.113900,0.113800,19.000000
4,No log,2.528494,0.136300,0.044700,0.112900,0.112800,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=248, training_loss=3.0255282002110637, metrics={'train_runtime': 282.6103, 'train_samples_per_second': 13.998, 'train_steps_per_second': 0.878, 'total_flos': 1070824333246464.0, 'train_loss': 3.0255282002110637, 'epoch': 4.0})

In [ ]:
trainer.save_model("my_fine_tuned_t5_small_model")

In [ ]:
model.save_pretrained("C:/Users/sathv/Desktop/publicPulse/publicPulse/backend")

save to drive


In [ ]:
model.save_pretrained("/content/drive/My Drive/fine-tuned-model")

In [ ]:
text = billsum['test'][100]['text']
text = "summarize: " + text
text

'summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 25417.5 of the Public Resources Code is amended to read:\n25417.5.\n(a) In furtherance of the purposes of the commission as set forth in this chapter, the commission has the power and authority to do all of the following:\n(1) Borrow money, for the purpose of obtaining funds to make loans pursuant to this chapter, from the California Economic Development Financing Authority, the California Infrastructure and Economic Development Bank, and the California Consumer Power and Conservation Financing Authority from the proceeds of revenue bonds issued by any of those agencies.\n(2) Pledge collateral to secure the repayment of moneys borrowed pursuant to paragraph (1) or of bonds or other borrowings by the California Infrastructure and Economic Development Bank. The commission may pledge, as collateral for these purposes, the loans made pursuant to this chapter or former Chapter 5.4 (commencing wit

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred

Token indices sequence length is longer than the specified maximum sequence length for this model (1027 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Section 25417.5 of the Public Resources Code is amended to read: 25421. The commission has the power and authority to do all of the following: (1) Borrow money, for the purpose of obtaining funds to make loans pursuant to this chapter, from the California Economic Development Financing Authority, the California Infrastructure and Economic Development Bank, and the California Consumer Power and Conservation Financincing Authority from the proceeds of revenue bonds issued by any of those agencies. The commission may pledge, as collateral for these purposes, the loans made pursuant . to the .'}]

In [ ]:
pred[0]['summary_text']

'Section 25417.5 of the Public Resources Code is amended to read: 25421. The commission has the power and authority to do all of the following: (1) Borrow money, for the purpose of obtaining funds to make loans pursuant to this chapter, from the California Economic Development Financing Authority, the California Infrastructure and Economic Development Bank, and the California Consumer Power and Conservation Financincing Authority from the proceeds of revenue bonds issued by any of those agencies. The commission may pledge, as collateral for these purposes, the loans made pursuant . to the .'

In [ ]:
preds = [pred[0]['summary_text']]

In [ ]:
labels = [billsum['test'][100]['summary']]

In [ ]:
rouge.compute(predictions=preds, references=labels, use_stemmer=True)

{'rouge1': 0.4049844236760125,
 'rouge2': 0.1128526645768025,
 'rougeL': 0.21806853582554517,
 'rougeLsum': 0.2741433021806854}

In [ ]:
def getVerdict(reviews):
  reviews=".".join(reviews)
  verdict = summarizer(reviews)
  return verdict[0]['summary_text']

# 3. RATING THE PRODUCT USING BERT MODEL

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt', truncation=True)

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

4

In [ ]:
def getRating(reviews):
  reviews=".".join(reviews)
  tokens = tokenizer.encode(reviews, return_tensors='pt', truncation=True)
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

# OUPUT FUNCTION FOR GETTING JSON OBJECT

In [ ]:
def getAll(url):
  json_obj=dict()
  rev_arr = getReviews(url)
  json_obj['rev_arr']=rev_arr
  json_obj['verdict']=getVerdict(rev_arr)
  json_obj['rating']=getRating(rev_arr)
  return json_obj

In [ ]:
abc = getReviews('https://www.yelp.com/biz/mejico-sydney-2')

In [ ]:
abc

['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
 'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to check on us.. I

In [ ]:
efg = getVerdict(abc)
efg

"The guacamole was bland and came with some type of plantain chips. The chicken and steak tacos were good. But the service was poor. We had a waitress with an attitude. She spilled the fixings for the less than average, but it wasn't fast enough for her. We went with Margarita's - they have $10 margaritas on Mondays (or perhaps all weekdays)"

In [ ]:
hij = getRating(abc)
hij

3

In [ ]:
klm = getAll('https://www.yelp.com/biz/mejico-sydney-2')

In [ ]:
klm

{'rev_arr': ['The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.',
  'The food was decent not great..  We had the guacamole which was bland and came with some type of plantain chips.. The chicken and steak tacos were good.. But the service was poor. We had a waitress with an attitude. She seemed upset whenever we asked for anything.  She would walk by and just stick up her hand and say " just wait ".  She spilled the ingredients to make the guacamole all over the table but never apologized. The waitress didn\'t come by at all, not even once to ch